In [1]:
import random
random.seed(42)
import numpy as np
import pandas as pd

from scipy.stats import invwishart
from numpy import sum, sqrt, outer, transpose
from numpy.random import multivariate_normal
from scipy.sparse import coo_matrix
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error as MSE

In [2]:
rating = pd.read_csv("rating.csv")
rating = rating.sample(10000)
rating.describe()

,userId,movieId,rating
count,10000.000000,10000.000000,10000.000000
mean,68355.960000,8891.222400,3.540800
std,39991.295438,19451.421605,1.060046
min,12.000000,1.000000,0.500000
25%,33698.250000,879.000000,3.000000
50%,68233.000000,2174.000000,4.000000
75%,102443.250000,4718.000000,4.000000
max,138486.000000,120783.000000,5.000000


In [3]:
# EDA
...

In [4]:
rating_matrix = rating.pivot(index='userId', columns='movieId', values='rating')
rating_matrix = rating_matrix.fillna(0)
R = rating_matrix.to_numpy()

In [5]:
# hyperparams: \Theta_0 = {\mu_0, T_0, \nu_0, S_0, sigma}
# \Theta_U = {\mu_U, \Sigma_U}
# \Theta_V = {\mu_V, \Sigma_V}

# Sample \Theta_U, \Theta_V
# Sample U, V
# Compute R

In [6]:
# Initialization
M = rating_matrix.shape[0] # number of customers/users
N = rating_matrix.shape[1] # number of movies
D = 10 # number of latent features
U = np.ones((D, M))
V = np.ones((D, N))

mu_0, T_0 = np.zeros((D, 1)), np.eye(D)
nu_0, S_0 = D, np.eye(D)
sigma = 1

In [7]:
nonzero_indices = R.nonzero()
nonzero_values = R[nonzero_indices]

In [8]:
iters = 100

Sigma_u, Sigma_v = np.eye(D), np.eye(D)

for it in range(iters):
    # Sample \Theta_U
    mu_ustar = inv(inv(T_0) + M*inv(Sigma_u))@(inv(T_0)@mu_0 + inv(Sigma_u)@sum(U, axis=1, keepdims=True))
    T_ustar = inv(inv(T_0) + M*inv(Sigma_u))
    mu_u = multivariate_normal(np.squeeze(np.asarray(transpose(mu_ustar))), T_ustar, 1)
    
    nu_ustar = nu_0 + M
    S_ustar = S_0 + (U - transpose(mu_u))@transpose(U - transpose(mu_u))
    inv_wishart = invwishart(nu_ustar, S_ustar)
    Sigma_u = inv_wishart.rvs()
    
    # Sample \Theta_V
    mu_vstar = inv(inv(T_0) + N*inv(Sigma_v))@(inv(T_0)@mu_0 + inv(Sigma_v)@sum(V, axis=1, keepdims=True))
    T_vstar = inv(inv(T_0) + N*inv(Sigma_v))
    mu_v = multivariate_normal(np.squeeze(np.asarray(transpose(mu_vstar))), T_vstar, 1)
    
    nu_vstar = nu_0 + N
    S_vstar = S_0 + (V - transpose(mu_v))@transpose(V - transpose(mu_v))
    inv_wishart = invwishart(nu_vstar, S_vstar)
    Sigma_v = inv_wishart.rvs()
    
    # Sample U
    for i in range(M):
        Lambda_ustar = inv(Sigma_u)
        theta_ustar = transpose(inv(Sigma_u)@transpose(mu_u))
        for j in range(N):
            if R[i, j] == 0:
                continue
            V_j = transpose(V[:, j])
            Lambda_ustar += outer(V_j, V_j)/sigma
            theta_ustar += V_j*R[i, j]/sigma
        theta_ustar = inv(Lambda_ustar)@transpose(theta_ustar)
        U[:, i] = multivariate_normal(np.squeeze(np.asarray(transpose(theta_ustar))), inv(Lambda_ustar), 1)
        
    # Sample V
    for j in range(N):
        Lambda_vstar = inv(Sigma_v)
        theta_vstar = transpose(inv(Sigma_v)@transpose(mu_v))
        for i in range(M):
            if R[i, j] == 0:
                continue
            U_i = transpose(U[:, i])
            Lambda_vstar += outer(U_i, U_i)/sigma
            theta_vstar += U_i*R[i, j]/sigma
        theta_vstar = inv(Lambda_vstar)@transpose(theta_vstar)
        V[:, j] = multivariate_normal(np.squeeze(np.asarray(transpose(theta_vstar))), inv(Lambda_vstar), 1)

    R_star = transpose(U)@V
    predicted_values = R_star[nonzero_indices]
    rmse = np.sqrt(MSE(nonzero_values, predicted_values))
    
    if it%10 == 0:
        print("Iter:", it, "RMSE:", rmse)

Iter: 0 RMSE: 6.289047926368645
Iter: 10 RMSE: 0.9503222612858223
Iter: 20 RMSE: 0.9612996186025389
Iter: 30 RMSE: 0.9573116225035936
Iter: 40 RMSE: 0.9571418937550362
Iter: 50 RMSE: 0.9461383389462567
Iter: 60 RMSE: 0.9474534225556884
Iter: 70 RMSE: 0.9492083094423441
Iter: 80 RMSE: 0.9392938877974769
Iter: 90 RMSE: 0.9427310406324433
